In [1]:
import jupyros as jr
import rospy
import tf
import ipywidgets as widgets
import math

from geometry_msgs.msg import PoseStamped
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

from robotcontroller import RobotController

rospy.init_node('jupyter_cmdvel_node')

In [2]:
jr.publish('/cmd_vel', Twist)

In [3]:
mapframe = 'odom'
transformlistener = tf.TransformListener()
cmdvelpub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

robotcontroller = RobotController(transformlistener, mapframe, cmdvelpub)
jr.subscribe('/odom', Odometry, robotcontroller.newodometry)

In [4]:
output = widgets.Output()

def slider_change(change):
    with output:
        newvalue = change['new']
        print(newvalue)
        robotcontroller.appendbehavior(RotateToState(newvalue))
    return

slider = widgets.FloatSlider(
    value=1,
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Target theta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
slider.observe(slider_change, names='value')

display(slider, output)

FloatSlider(value=1.0, continuous_update=False, description='Target theta:', max=3.141592653589793, min=-3.141…

Output()

In [5]:
from rotatetostate import RotateToState
from drivetoposition import DriveToPosition
from nav_msgs.msg import Path
from map import Map
import traceback

mapframe = 'odom'

map = Map()

pathpub = rospy.Publisher('pythonpath', Path, queue_size=10)

def navgoal(message):

    if robotcontroller.latestodominmapframe:

        navgoalinframe = transformlistener.transformPose(mapframe, message)

        robotx = robotcontroller.latestodominmapframe.pose.position.x
        roboty = robotcontroller.latestodominmapframe.pose.position.y

        dx = navgoalinframe.pose.position.x - robotx
        dy = navgoalinframe.pose.position.y - roboty

        yaw = math.atan2(dy, dx)

        try:
            rospy.loginfo('Planning path...')
            
            p = map.findpath(robotcontroller.latestodominmapframe, navgoalinframe)
            if p is not None:
                path = Path()
                path.header.frame_id = 'map'
                path.header.stamp = rospy.Time.now()

                for po in p:
                    (x, y) = po
                    pose = PoseStamped()
                    pose.pose.position.x = map.latestmap.info.origin.position.x + x * map.latestmap.info.resolution
                    pose.pose.position.y = map.latestmap.info.origin.position.y + y * map.latestmap.info.resolution
                    path.poses.append(pose)

                rospy.loginfo('Path calculated with %s poses', len(path.poses))
                pathpub.publish(path)
                             
                x = len(p) - 1
                while x >= 1:
                    (xtarget, ytarget) = p[x]
                    (xorigin, yorigin) = p[x - 1]
                    
                    pose = PoseStamped()
                    pose.pose.position.x = map.latestmap.info.origin.position.x + xtarget * map.latestmap.info.resolution
                    pose.pose.position.y = map.latestmap.info.origin.position.y + ytarget * map.latestmap.info.resolution
                    path.poses.append(pose)

                    robotcontroller.appendbehavior(DriveToPosition(pose))

                    dx = xtarget - xorigin
                    dy = ytarget - yorigin

                    yaw = math.atan2(dy, dx)
                    robotcontroller.appendbehavior(RotateToState(yaw))
                    
                    x = x -1
                    
                # Calculate to first waypoint
                (xtarget, ytarget) = p[1]
                (xorigin, yorigin) = p[0]
                
                pose = PoseStamped()
                pose.pose.position.x = map.latestmap.info.origin.position.x + xtarget * map.latestmap.info.resolution
                pose.pose.position.y = map.latestmap.info.origin.position.y + ytarget * map.latestmap.info.resolution
                path.poses.append(pose)

                robotcontroller.appendbehavior(DriveToPosition(pose))

                dx = xtarget - xorigin
                dy = ytarget - yorigin

                yaw = math.atan2(dy, dx)
                robotcontroller.appendbehavior(RotateToState(yaw))

                rospy.loginfo('Navigation state chain computed')

            else:
                rospy.loginfo('No path found')
                
        except Exception as e:
            rospy.loginfo('Error creating path : %s', traceback.format_exc())

    return

jr.subscribe('/move_base_simple/goal', PoseStamped, navgoal)

In [6]:
from nav_msgs.msg import OccupancyGrid
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=300, height=500)
display(canvas)

def newnavgrid(message):

    try:
        map.newmap(message)

        with hold_canvas():
            canvas.clear()
            canvas.layout.width = str(message.info.width) + 'px'
            canvas.layout.height = str(message.info.height) + 'px'
            canvas.width = message.info.width
            canvas.height = message.info.height

            if True:
                for y in range(message.info.height):
                    for x in range(message.info.width):
                        bottomx = x;
                        bottomy = message.info.height - 1 - y

                        value = message.data[y * message.info.width + x]

                        if value == 0:
                            # Free cell
                            canvas.fill_style = "lightgray"
                            canvas.stroke_style = "lightgray"

                            canvas.fill_rect(bottomx, bottomy - 1, 1, 1)
                        if value > 0:
                            # Occupied
                            canvas.fill_style = "black"
                            canvas.stroke_style = "black"

                            canvas.fill_rect(bottomx, bottomy - 1, 1, 1)

            if robotcontroller.latestodominmapframe:
                robotx = robotcontroller.latestodominmapframe.pose.position.x
                roboty = robotcontroller.latestodominmapframe.pose.position.y

                mapx = int((robotx - message.info.origin.position.x) / message.info.resolution)
                mapy = int((roboty - message.info.origin.position.y) / message.info.resolution)

                canvas.fill_style = "red"
                canvas.stroke_style = "red"

                bottomy = message.info.height - 1 - mapy
                canvas.fill_rect(mapx, bottomy + 1, 4, 4)

    except Exception as e:
        rospy.logerr('Error processing map data : %s', e)


jr.subscribe('/costmap_node/costmap/costmap', OccupancyGrid, newnavgrid)

Canvas(width=300)